<a href="https://colab.research.google.com/github/TharunShreeB/object-detection-speed-tracking-/blob/main/object_tracking_speed_detection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Setup YOLOv5 and dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -q opencv-python matplotlib seaborn tqdm
!pip install -q torch torchvision

import torch
import cv2
import numpy as np
import time
import uuid
from IPython.display import display, clear_output
from google.colab.patches import cv2_imshow
from google.colab import files

# STEP 2: Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)

# STEP 3: Upload video file
uploaded = files.upload()
video_path = next(iter(uploaded))

# STEP 4: Constants & video setup
PIXEL_TO_METER = 0.5  # <-- Calibrate this properly!
cap = cv2.VideoCapture(video_path)
FPS = cap.get(cv2.CAP_PROP_FPS)

vehicle_positions = {}  # {vehicle_id: (x, y)}

out = cv2.VideoWriter('output.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      FPS,
                      (int(cap.get(3)), int(cap.get(4))))

frame_count = 0

# STEP 5: Helper function for ID tracking
def assign_ids(prev_positions, curr_positions, max_distance=50):
    new_tracking = {}
    used = set()

    for prev_id, (px, py) in prev_positions.items():
        closest_id = None
        closest_dist = max_distance
        for curr_id, (cx, cy) in curr_positions.items():
            if curr_id in used:
                continue
            dist = np.linalg.norm(np.array([cx, cy]) - np.array([px, py]))
            if dist < closest_dist:
                closest_id = curr_id
                closest_dist = dist

        if closest_id is not None:
            new_tracking[closest_id] = (curr_positions[closest_id], prev_id)
            used.add(closest_id)

    # Assign new IDs to unmatched detections
    for curr_id, (cx, cy) in curr_positions.items():
        if curr_id not in used:
            new_tracking[curr_id] = ((cx, cy), str(uuid.uuid4())[:8])

    # Return: {vehicle_id: position}
    return {vehicle_id: position for (_, (position, vehicle_id)) in new_tracking.items()}

# STEP 6: Main loop for detection and speed estimation
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    current_raw = {}

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) != 2:  # Only detect class 2 = car
            continue

        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        current_raw[f"{cx}-{cy}"] = (cx, cy)

    # Match vehicles across frames
    tracked_positions = assign_ids(vehicle_positions, current_raw)

    # Draw boxes and display speeds
    for vid, (cx, cy) in tracked_positions.items():
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            if int(cls) != 2:
                continue
            box_cx = int((x1 + x2) / 2)
            box_cy = int((y1 + y2) / 2)
            if abs(box_cx - cx) < 5 and abs(box_cy - cy) < 5:
                break

        if vid in vehicle_positions:
            px, py = vehicle_positions[vid]
            dist_pixels = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)
            speed_mps = dist_pixels * PIXEL_TO_METER * FPS
            speed_kmph = speed_mps * 3.6
        else:
            speed_kmph = 0

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"{int(speed_kmph)} km/h", (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    vehicle_positions = tracked_positions
    out.write(frame)

    if frame_count % 10 == 0:
        clear_output(wait=True)
        cv2_imshow(frame)

cap.release()
out.release()

# STEP 7: Download output
files.download('output.avi')

Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17326 (from 3)
Receiving objects: 100% (17410/17410), 16.30 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (11934/11934), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s 